In [1]:
import os
import random
import string
import gdal
import math
from itertools import product
from itertools import starmap
import shapefile
import numpy as np
import pandas as pd
import pyproj
import shapely.geometry
import spatial_csv_to_kml
from rasterstats import zonal_stats

from rtree import index

from shapely.geometry import Polygon
from shapely.geometry import Point

import multistage_sampling 

/Users/jiehuachen/.virtualenvs/sampling/lib/python2.7/site-packages/pandas/io/excel.py:626: UserWarning: Installed openpyxl is not supported at this time. Use >=1.6.1 and <2.0.0.
  .format(openpyxl_compat.start_ver, openpyxl_compat.stop_ver))


### Getting Geosurvey data: those data are shared through dropbox link: https://www.dropbox.com/s/zrq0bx83dgzpvsp/data.zip?dl=0


In [2]:
shpfile = "data/tansis_sampling_districts_laea.shp"
inputfile = "data/TZ_mean_10k.tif"
cmd = "gdalwarp -cutline " + shpfile + " -crop_to_cutline -srcnodata \"nan\" -dstnodata \"nan\" " + inputfile + " mean_output.tif"
os.system(cmd)

inputfile = "data/TZ_crp_ens_1k.tif"
cmd = "gdalwarp -cutline " + shpfile + " -crop_to_cutline -srcnodata \"nan\" -dstnodata \"nan\" " + inputfile + " mean_output_1k.tif"
os.system(cmd)


256

In [4]:
crpdat = gdal.Open("mean_output.tif", gdal.GA_ReadOnly)
crp_prob_np = np.asarray(crpdat.GetRasterBand(1).ReadAsArray())
originX, pixelWidth, rx, originY, ry, pixelHeight = crpdat.GetGeoTransform()

crpdat_1k = gdal.Open("mean_output_1k.tif", gdal.GA_ReadOnly)
crp_prob_np_1k = np.asarray(crpdat.GetRasterBand(1).ReadAsArray())
originX_1k, pixelWidth_1k, rx_1k, originY_1k, ry_1k, pixelHeight_1k = crpdat.GetGeoTransform()

In [5]:
cutoff_mean = 0.7
cutoff_sd = 0.1
crp_presence_loc = np.where((crp_prob_np>cutoff_mean))
n_presence = crp_presence_loc[0].shape[0]
n_presence

1080

In [6]:
def getcoords(idxx, idxy, w=pixelWidth, h=pixelHeight, x0 = originX, y0=originY):
    x = x0 + w * idxy
    y = y0 + h * idxx
    return [x,y]

crp_presence_coords = list(starmap(getcoords, zip(*crp_presence_loc)))

In [7]:
shpfile = "data/tansis_sampling_districts_laea.shp"
districts_roi_shp = shapefile.Reader(shpfile, 'rb')
districts_roi_names = map(lambda x: x[6], districts_roi_shp.records())

### Find locations within each district, which have cropland presence probability larger than the cutoff value

In [8]:
regions = districts_roi_shp.shapes()
square_size = 10000
progress_mark = 500

In [9]:
points_with_regions = []

rtree_idx = index.Index()

for i,r in enumerate(regions):
    rtree_idx.insert(i,r.bbox)
    
n = len(crp_presence_coords)
locs_inregion = []
for k,georef in enumerate(crp_presence_coords):
    if 0 == k % progress_mark: print "{}/{} ~= {:.2f}%% complete".format(k,n,100*float(k)/n)
    x,y = georef
    square = shapely.geometry.geo.box(x, y-square_size, x+square_size, y)
    for j in rtree_idx.intersection((x,y-square_size,x+square_size,y)):
        if shapely.geometry.asShape(regions[j]).contains(square):
            points_with_regions.append([x,y,j]) 
            break # WARNING: this assumes exactly one region will contain the point

0/1080 ~= 0.00%% complete
500/1080 ~= 46.30%% complete
1000/1080 ~= 92.59%% complete


In [10]:
points_with_regions_pd = pd.DataFrame(zip(*points_with_regions)).transpose()
points_with_regions_pd.columns = ['x', 'y','district_idx']
points_with_regions_pd.to_csv("locations_crop_10k.csv", index=False)

### Find high cropland presence grids and their locations

In [11]:
points_with_regions_pd = pd.DataFrame(zip(*points_with_regions)).transpose()
points_with_regions_pd.columns = ['x','y','district']
district_highcrp = map(lambda x: int(x), points_with_regions_pd.groupby('district').count().index)
district_highcrp_count = points_with_regions_pd.groupby('district').count()
crp_area_prob = [district_highcrp_count[district_highcrp_count.index==k].x.get_values()[0] if k in district_highcrp else 0 for k in xrange(len(districts_roi_names))]
crp_area_prob = map(lambda x: x*1.0/sum(crp_area_prob), crp_area_prob)


### sample size

In [12]:
n_10k = 200
n_1k = 4
n_100m = 5
n_10k_perdistrict = map(lambda x: int(math.ceil(n_10k*x)), crp_area_prob)
sum(n_10k_perdistrict)

231

### Start Sampling: the results are sampling results of csv files for each district, csv, kml files for each 10k-by-10k, kml files for drone flights organized in the corresponding district folder

In [28]:
random.seed(20150906)
reload(multistage_sampling)

<module 'multistage_sampling' from 'multistage_sampling.pyc'>

In [29]:
for k, sample_n_10k in enumerate(n_10k_perdistrict):
    if sample_n_10k >0:
        district_locs = points_with_regions_pd[points_with_regions_pd.district==k]
        sampled_locs_idx = random.sample(list(district_locs.index), sample_n_10k)
        sampled_locs =  district_locs.ix[sampled_locs_idx]
        x_current_lower = list(sampled_locs.x.get_values()+5000)
        y_current_lower = list(sampled_locs.y.get_values()-5000)
        multistage_sampling.sample(x_current_lower,  y_current_lower, n_100m, districts_roi_names[k], "mean_output_1k.tif")
    
        sampled_data = pd.read_csv('output/'+districts_roi_names[k]+'/'+districts_roi_names[k]+'_'+str(0)+'.csv')[['y','x']]
        total_file = 'output/'+districts_roi_names[k]+'/'+districts_roi_names[k]+'.csv'
        sampled_data.to_csv(total_file, index=False)
        if sample_n_10k>1:
            with open(total_file, 'a') as f:            
                for s in xrange(1, sample_n_10k):
                    filename = 'output/'+districts_roi_names[k]+'/'+districts_roi_names[k]+'_'+str(s)+'.csv'
                    sampled_data = pd.read_csv(filename)[['y','x']]
                    sampled_data.to_csv(f, header=False, index=False)
            
        filenames_gpx = [os.system('rm '+'"output/'+districts_roi_names[k]+'/'+districts_roi_names[k]+'_'+str(s)+'_Waypoints'+'.csv"') for s in xrange(sample_n_10k)]
        


[4, 4, 4, 4]
output/Arumeru/Arumeru_0
0
E1873-S911
1
E1873-S912
2
E1880-S906
3
E1874-S906
4
E1875-S914
5
E1873-S906
6
E1875-S915
7
E1873-S908
8
E1878-S907
9
E1880-S914
10
E1879-S915
11
E1880-S910
12
E1880-S911
13
E1876-S906
[4, 4, 4, 4]
output/Arumeru/Arumeru_1
0
E1861-S908
1
E1864-S915
2
E1867-S906
3
E1866-S907
4
E1868-S907
5
E1868-S911
6
E1868-S915
7
E1861-S911
8
E1862-S911
9
E1863-S913
10
E1869-S908
11
E1870-S913
12
E1865-S906
13
E1861-S906
14
E1869-S912
[4, 4, 4, 4]
output/Arumeru/Arumeru_2
0
E1877-S916
1
E1877-S922
2
E1871-S920
3
E1873-S925
4
E1873-S924
5
E1874-S916
6
E1878-S925
7
E1877-S919
8
E1880-S918
9
E1872-S920
10
E1878-S921
11
E1875-S923
12
E1876-S920
13
E1875-S920
14
E1875-S925
15
E1878-S923
[4, 4, 4, 4]
output/Karatu_Arusha/Karatu_Arusha_0
0
E1738-S921
1
E1732-S921
2
E1732-S920
3
E1730-S919
4
E1732-S916
5
E1732-S917
6
E1736-S920
7
E1734-S921
8
E1734-S922
9
E1739-S921
10
E1735-S923
11
E1736-S918
12
E1738-S924
[4, 4, 4, 4]
output/Karatu_Arusha/Karatu_Arusha_1
0
E1749-S911
1

In [30]:
os.chdir("output/")
import glob
locs_csvfiles =  glob.glob("*")
locs_data = pd.read_csv(os.path.join(locs_csvfiles[0],locs_csvfiles[0]+".csv"))
locs_data['district_name'] = locs_csvfiles[0]
locs_data.to_csv("sampled_locs_total.csv", index=False)
with open("sampled_locs_total.csv", 'a') as f: 
    for locs_csvfile in locs_csvfiles[1:]:
        sampled_data = pd.read_csv(os.path.join(locs_csvfile, locs_csvfile+".csv"))
        sampled_data['district_name'] = locs_csvfile
        tmp = [locs_data, sampled_data]
        locs_data = pd.concat(tmp)
        sampled_data.to_csv(f, header=False, index=False)